In [1]:
import numpy as np
import pandas as pd
import pickle as pk
data = pd.read_csv('data/tcd ml 2019-20 income prediction training (with labels).csv')

# Preview and visualise data

In [2]:
data.head(10)

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.16217
5,6,1991.0,male,60.0,Liberia,56080,senior security analyst,Bachelor,0,Blond,181,125809.99030
6,7,1982.0,NaN,71.0,State of Palestine,1205185,program associate,Bachelor,0,Black,174,150319.42260
7,8,2008.0,male,36.0,Israel,31419,retail specialist,NaN,0,Brown,190,11849.02876
8,9,2015.0,male,43.0,South Sudan,85333,intergovernmental outreach coordinator,No,0,Blond,189,48058.20607
9,10,2019.0,female,16.0,Cuba,1325383,lease analyst,Bachelor,1,Black,157,90940.68912


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111993 entries, 0 to 111992
Data columns (total 12 columns):
Instance             111993 non-null int64
Year of Record       111552 non-null float64
Gender               104561 non-null object
Age                  111499 non-null float64
Country              111993 non-null object
Size of City         111993 non-null int64
Profession           111671 non-null object
University Degree    104623 non-null object
Wears Glasses        111993 non-null int64
Hair Color           104751 non-null object
Body Height [cm]     111993 non-null int64
Income in EUR        111993 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 10.3+ MB


In [4]:
data['Wears Glasses'].value_counts()

1    56056
0    55937
Name: Wears Glasses, dtype: int64

In [5]:
data.describe()

,Instance,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR
count,111993.000000,111552.000000,111499.000000,1.119930e+05,111993.000000,111993.000000,1.119930e+05
mean,55997.000000,1999.421274,37.345304,8.388538e+05,0.500531,175.220192,1.092138e+05
std,32329.738686,11.576382,16.036694,2.196879e+06,0.500002,19.913889,1.498024e+05
min,1.000000,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03
25%,27999.000000,1989.000000,24.000000,7.273400e+04,0.000000,160.000000,3.077169e+04
50%,55997.000000,1999.000000,35.000000,5.060920e+05,1.000000,174.000000,5.733917e+04
75%,83995.000000,2009.000000,48.000000,1.184501e+06,1.000000,190.000000,1.260936e+05
max,111993.000000,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06


In [6]:
corr_matrix = data.corr()
corr_matrix['Income in EUR']

Instance            0.002897
Year of Record      0.165116
Age                 0.186160
Size of City        0.014993
Wears Glasses       0.005718
Body Height [cm]    0.072889
Income in EUR       1.000000
Name: Income in EUR, dtype: float64

In [7]:
data = pd.read_csv('data/tcd ml 2019-20 income prediction training (with labels).csv')
data['Profession'].value_counts()

pipefitter                               261
procedural justice coordinator           256
production support                       246
port marine engineer                     246
preplacement nurse practitioner          245
                                        ... 
audit engineer                             1
computer aide                              1
audit supervisor                           1
accessibility outreach coordinator         1
assistant business services associate      1
Name: Profession, Length: 1340, dtype: int64

# Prepare data for ml algorithm

In [8]:
data = pd.read_csv('data/tcd ml 2019-20 income prediction training (with labels).csv')
data = data.fillna(value=0)

## 'Instance'
data = data.drop(labels='Instance', axis=1)

## 'Year of Record'
# nothing to do here

## 'Gender'
data = data.replace(to_replace={'Gender': {
        'male': 1,
        'female': -1,
        'other': 0,
        'unknown': 0,
        '0': 0,
        np.nan: 0,
}})
# data['Gender'] = data['Gender'].astype(float)

## 'Age'
# nothing to do here

## 'Country'
## 'Size of City'
## 'Profession'

jobs = pd.Series.to_dict(data['Profession'].value_counts())
new_jobs = {'Profession': {}}

for key in jobs:
    if jobs[key] > 80:
        try:
            new_key = key[:4]
        except TypeError:
            new_key = 'unkn'
    else:
        new_key = 'unkn'
    new_jobs['Profession'][key] = new_key
    
for key in new_jobs['Profession']:
    try:
        temp = new_jobs['Profession'][new_jobs['Profession'][key]]
        new_jobs['Profession'][key] = 'unkn'
    except KeyError:
        continue

# for key in new_jobs['Profession']:
#     print(key, '->', new_jobs['Profession'][key], ' * ', type(key))

pk.dump(new_jobs, open('jobs.pkl', mode='wb'))
data = data.replace(to_replace=new_jobs)
data.head()

## 'University Degree'
data = data.replace(to_replace={'University Degree': {
        '0': 'No',
}})

## 'Wears Glasses'
## 'Hair Color'
data = data.replace(to_replace={'Hair Color': {
        '0': 'Unknown',
}})

## 'Body Height [cm]'
## 'Income in EUR'

In [9]:
data['Profession'].value_counts()

unkn    17907
seni    10405
prog     2815
qual     2170
publ     2030
        ...  
hro        82
hous       82
spen       82
hunt       82
impl       81
Name: Profession, Length: 227, dtype: int64

In [10]:
# data.head(15)

In [11]:
data = pd.get_dummies(data, columns=['Country', 'Profession', 'University Degree', 'Hair Color'], sparse=True)
data = data.astype(float)

In [12]:
# data.info()
# corr_matrix = data.corr()
# corr_matrix['Income in EUR']

In [13]:
from sklearn.preprocessing import scale
X = data.drop(columns=['Income in EUR']).to_numpy()
y = data.loc[:, 'Income in EUR'].to_numpy()

Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12120)

# Train algorithm

In [ ]:
from sklearn.linear_model import LinearRegression # N/A
from sklearn.tree import DecisionTreeRegressor # 0.5144117272297519
from sklearn.ensemble import RandomForestRegressor # 0.6767255620150188
from sklearn.neighbors import KNeighborsRegressor # -0.15649302917615013

func = RandomForestRegressor()#verbose=True)
func.fit(X_train, y_train)
func.score(X_test, y_test)

/usr/local/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
func.predict(X[:5])

In [ ]:
pk.dump(func, open('dump.pkl', mode='wb'))